# Recherche de lyrics par artiste

Notre but est de récupérer par artiste, l'ensemble des paroles de ses sons.


__ATTENTION__ ici je ne me concentre que sur les albums des artistes, c'est à dire que tous les song qui ne sont pas sorties dans un album ne sont pas comtabilisé. En effet, sans cette condition difficile de mettre une année sur la diffusion du son.


Nous allons utiliser ce site [https://search.azlyrics.com](https://search.azlyrics.com) pour la recherche des paroles.
<br>
<br>

Voici donc les étapes de la constitution de cette base:

-  Dans un premier temps, la fonction <a href='#artist_to_url'>artist_to_url</a> nous permet de générer à l'aide d'un nom d'artiste un url vers l'ensemble de sa discographie.
-  Notre deuxième fonction <a href='#album_url_year'>album_url_year</a> va nous permettre de ressortir JE NE SAIS PAS ENCORE SOUS QUEL FORMAT le nom de l'artiste, le nom de l'album, l'année de difusion et enfin les liens vers les musiques contenus dans les albums.


In [1]:
import pandas as pd
import numpy as np
import re
import unidecode
import requests
from bs4 import BeautifulSoup
import urllib
import urllib.request
from lxml import html
import sys

# from nltk.stem.snowball import EnglishStemmer
# from nltk.stem.snowball import SnowballStemmer
# from nltk.tokenize import TreebankWordTokenizer

from random_user_agent.user_agent import UserAgent
from random_user_agent.params import SoftwareName, OperatingSystem
import warnings

In [2]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Wenceslas\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [3]:
#permet de générer l'url pour accéder à l'ensemble des titres de l'artiste
#certain artiste comme Renaud ne sont pas référencé
def artist_to_url(artist):
    """ A partir d'un nom d'artiste, on produit un URL pour la connexion vers le site 
        `https://www.azlyrics.com`. 
        
    On a remarqué qu'il existait un pattern pour
    générer un URL à partir d'un artiste.
    En effet, si le premier caractère du nom est une lettre, alors l'URL se construit
    de la façon suivante:
        https://www.azlyrics.com/{première lettre du nom de l'artiste}/{le nom de l'artiste}
    Sinon:
        https://www.azlyrics.com/19/{le nom de l'artiste}
        
    Args:
        -artist: désigne le nom de l'artiste
    
    Résultat:
        >>> artist_to_url("Schoolboy Q")
        >>> https://www.azlyrics.com/s/schoolboyq.html'
    
    Raises:
        None
        
    """
    url_base= """https://www.azlyrics.com"""
    name= re.sub(' ','',artist)
    # Enlever les accents comme é ou â
    name= unidecode.unidecode(name.lower())
    first_l= list(name)[0]
    # Certains artistes ont un nom qui commence par un chiffre
    # dans ce cas, la procédure de création de l'url est un peu différent
    if first_l.isalpha():
        url= """{}/{}/{}.html""".format(url_base, first_l, name)
    else:
        url= """{}/19/{}.html""".format(url_base, name)
    return url

In [4]:
#test
artist_to_url("Schoolboy Q")

'https://www.azlyrics.com/s/schoolboyq.html'

In [5]:
def transform_proxy_http(proxy):
    """ A partir d'un proxy (avec son port), on cherche à générer une adresse de connexion http
        
    Args:
        -proxy
        
    Résultat:
        >>> transform_proxy_http("91.228.8.162:8080")
        >>> "http://91.228.8.162:8080"
        
    Raises:
        None
        
    """
    return "http://"+proxy

In [ ]:
# On récupère une liste de proxy (mise à jour quotidienne)
urll= "https://raw.githubusercontent.com/clarketm/proxy-list/master/proxy-list-raw.txt"
urllib.request.urlretrieve(urll, 'data/proxy_list.txt')

listed_proxy= []
f= open("data/proxy_list.txt", "r")
listed_proxy= f.read().split("\n")

In [ ]:
# On cherche à faire une pré-sélection des proxy 
# On teste donc leur connexion sur le site qu'on cherche à scaper

good_prox= []
# Pour raison de simpliciter, on cherche que les proxy qui ont un port 8080
pat= re.compile('.:8080$')

proxies_list= [l for l in listed_proxy \
               if l in list(filter(pat.findall, listed_proxy))]

for prox in proxies_list:
    # On cherche 20 proxy de bonnes qualités
    if len(good_prox) <= 20:
        try:
            print(prox)
            
            #On génère un User Agent aléatoire pour chaque proxy
            software_names = [SoftwareName.CHROME.value]
            operating_systems = [OperatingSystem.WINDOWS.value, OperatingSystem.LINUX.value]
            user_agent_rotator = UserAgent(software_names=software_names,
                           operating_systems=operating_systems, limit=100)
            
            proxies= {"http":transform_proxy_http(prox),
                    "https":transform_proxy_http(prox)}

            user_agent = user_agent_rotator.get_random_user_agent()
            headers= {"User-Agent":user_agent}

            r = requests.Session()
            r.headers.update(headers)
            r.proxies.update(proxies)

            # Connexion à la page
            page= r.get("https://www.azlyrics.com/", proxies= proxies, headers= headers)

            # Si la connexion est fructueuse, alors le proxy est stocké
            good_prox.append(prox)
        except:
            # Si je ne peux pas me connecter avec ce proxy, alors je teste le suivant
            print("Not Good")
            continue
    else:
        # Si j'ai 20 bon proxy, j'arrète la sélection
        break
print("Fin")

good_prox[:5]

In [11]:
def album_url_year(artist, proxies_list= None):
    """Permet de générer pour un artiste un dataframe contant les informations sur ses
    productions
    
    A partir du nom de l'artiste, cette fonction récupère tous les albums produit par l'artiste
    (qui sont disponibles sur le site https://www.azlyrics.com) auquel est associé 
    une année de publication et une liste d'URL menant vers les paroles des musiques de cet album.
    On laisse le choix à l'utilisateur d'utiliser une liste de proxy pour requêter.
        
    Args:
        -artist: le nom de l'artiste
        -proxies_list: une liste de proxy, utilise si on veut scraper une groose
            quantité d'artiste. Par défaut, cet argument prend la valeur `None` car le fait d'utiliser
            les proxy ralentit énormément le temps de requétage.
        
    Résultat:
        >>> album_url_year("jinjer")
        >>> 	Artiste	Annee	Album	Url
            0	jinjer	2012	Inhale. Do Not Breathe	https://www.azlyrics.com/lyrics/jinjer/untilth...
            1	jinjer	2012	Inhale. Do Not Breathe	https://www.azlyrics.com/lyrics/jinjer/waltz.html
            2	jinjer	2012	Inhale. Do Not Breathe	https://www.azlyrics.com/lyrics/jinjer/scissor...
            3	jinjer	2012	Inhale. Do Not Breathe	https://www.azlyrics.com/lyrics/jinjer/exposed...
            4	jinjer	2012	Inhale. Do Not Breathe	https://www.azlyrics.com/lyrics/jinjer/mylostc...
            5	jinjer	2014	Cloud Factory     	    https://www.azlyrics.com/lyrics/jinjer/outland...
            6	jinjer	2014	Cloud Factory    	    https://www.azlyrics.com/lyrics/jinjer/aplusor...
    
    Raises:
    """
    year, album_name, url_song= [], [], []
    
    
    if proxies_list:

        random_proxy= sorted(proxies_list, key=lambda x: random())
        i = 0
        for prox in random_proxy:
            if i < 5: #si trop de fail
                i += 1
                try:

                    proxies= {"http":transform_proxy_http(prox),
                            "https":transform_proxy_http(prox)}

                    user_agent = user_agent_rotator.get_random_user_agent()
                    headers= {"User-Agent":user_agent}

                    r = requests.Session()
                    r.headers.update(headers)
                    r.proxies.update(proxies)

                    page= r.get(artist_to_url(artist), proxies= proxies, headers= headers)

                    soup= BeautifulSoup(page.text, 'html.parser')
                    html_page= soup.find('div', id= 'listAlbum')

                    html_data= str(html_page).split('<div class="album"')[1:][:-1]
                    for ht in html_data:
                    #je défini mes patterns REGEX pour récupérer de la page html le nom des albums, avec son année et les url des lyrics menant aux paroles de chacunes des musiques de l'album     
                        pattern_yr= re.compile('</b> \((\d{4})\)</div>|</b> \((\d{4})\)<br/>')
                        pattern_alb= re.compile('<b>"([\s\S]*)"</b>')
                        pattern_url= re.compile('href="..([\s\S]*).html" target=')

                        year.append(pattern_yr.search(ht).group(1))
                        album_name.append(pattern_alb.search(ht).group(1))
                        #le traitement pour les url est un peu différent
                        url_list= ht.split('\n')
                        url_base= "https://www.azlyrics.com"
                        url_song.append([url_base + pattern_url.search(href).group(1) + ".html" \
                                for href in url_list \
                                if pattern_url.search(href) != None])
                    try:
                        df= pd.DataFrame(
                        {
                            "Artiste": [artist]*len(year),
                            "Annee": year,
                            "Album": album_name,
                            "Url": url_song
                        })
                    except:
                        warnings.warn("Attention, il y a un problème dans la construction du DataFrame")
                        return None

                    #le df que je creer contient en URF une liste des url, je veux donc avoir une ligne par URL et donc
                    #les valeurs annee album, et artiste qui correspondent.
                    df_not_listed= pd.DataFrame({
                      col:np.repeat(df[col].values, df["Url"].str.len())
                      for col in df.columns.drop("Url")
                    }
                    ).assign(**{
                        "Url":np.concatenate(df["Url"].values)
                    })[df.columns]

                    return df_not_listed
                except:
                    warnings.warn("Attention, le proxy {} n'a pas permis de vous connecter à \
                                  la page souhaitée".format(prox))
                    continue
            else:
                break
                
        warnings.warn("Attention, l'artiste {} n'a pas été trouvé".format(artist))
        return None
    
    else:
        page= requests.get(artist_to_url(artist)) #l'appelle la fonction précédente
        soup= BeautifulSoup(page.text, 'html.parser')
        html_page= soup.find('div', id= 'listAlbum')
        try:
            html_data= str(html_page).split('<div class="album"')[1:][:-1]
            for ht in html_data:
            #je défini mes patterns REGEX pour récupérer de la page html le nom des albums, avec son année et les url des lyrics menant aux paroles de chacunes des musiques de l'album     
                pattern_yr= re.compile('</b> \((\d{4})\)</div>|</b> \((\d{4})\)<br/>')
                pattern_alb= re.compile('<b>"([\s\S]*)"</b>')
                pattern_url= re.compile('href="..([\s\S]*).html" target=')

                year.append(pattern_yr.search(ht).group(1))
                album_name.append(pattern_alb.search(ht).group(1))
                #le traitement pour les url est un peu différent
                url_list= ht.split('\n')
                url_base= "https://www.azlyrics.com"
                url_song.append([url_base + pattern_url.search(href).group(1) + ".html" \
                        for href in url_list \
                        if pattern_url.search(href) != None])
            try:
                df= pd.DataFrame(
                {
                    "Artiste": [artist]*len(year),
                    "Annee": year,
                    "Album": album_name,
                    "Url": url_song
                })
                
            except:
                warnings.warn("Attention, il y a un problème dans la construction du DataFrame")
                return None

            #le df que je creer contient en URF une liste des url, je veux donc avoir une ligne par URL et donc
            #les valeurs annee album, et artiste qui correspondent.
            df_not_listed= pd.DataFrame({
              col:np.repeat(df[col].values, df["Url"].str.len())
              for col in df.columns.drop("Url")
            }
            ).assign(**{
                "Url":np.concatenate(df["Url"].values)
            })[df.columns]

            return df_not_listed
        
        except:
            warnings.warn("Attention, l'artiste {} n'a pas été trouvé".format(artist))
            return None

In [13]:
album_url_year?

In [7]:
album_url_year("jinjer")

,Artiste,Annee,Album,Url
0,jinjer,2012,Inhale. Do Not Breathe,https://www.azlyrics.com/lyrics/jinjer/untilth...
1,jinjer,2012,Inhale. Do Not Breathe,https://www.azlyrics.com/lyrics/jinjer/waltz.html
2,jinjer,2012,Inhale. Do Not Breathe,https://www.azlyrics.com/lyrics/jinjer/scissor...
3,jinjer,2012,Inhale. Do Not Breathe,https://www.azlyrics.com/lyrics/jinjer/exposed...
4,jinjer,2012,Inhale. Do Not Breathe,https://www.azlyrics.com/lyrics/jinjer/mylostc...
5,jinjer,2014,Cloud Factory,https://www.azlyrics.com/lyrics/jinjer/outland...
6,jinjer,2014,Cloud Factory,https://www.azlyrics.com/lyrics/jinjer/aplusor...
7,jinjer,2014,Cloud Factory,https://www.azlyrics.com/lyrics/jinjer/nohoard...
8,jinjer,2014,Cloud Factory,https://www.azlyrics.com/lyrics/jinjer/cloudfa...
9,jinjer,2014,Cloud Factory,https://www.azlyrics.com/lyrics/jinjer/whoisgo...


In [ ]:

artiste_name_style = {
    "Slipknot":"rock",
    "DREAMERS":"rock",
    "Bring Me The Horizon":"rock",
    "Motionless in White":"rock",
    "Falling In Reverse":"rock",
    "Eminem":"rap",
    "Justin Bieber":"pop",
    "Post Malone":"rap",
    "Billie Eilish":"pop",
    "Mac Miller":"rap",
}
artiste_name= [name for name in artiste_name_style.keys()]

artist_random= sorted(artist_name, key=lambda x: random())
df= pd.DataFrame()
for art in artist_random:
    print("\n"+art)
    df= pd.concat([album_url_year(art), df], ignore_index= True)

print("\nFin")

In [ ]:
#ajout colonne Style à partir du dictionnaire
df["Style"]= df["Artiste"].map(artiste_name_style)
df.head()

In [ ]:
def verification(df):
    print("Vérifier si des valeurs sont None")
    for col in df.columns:
        nul= (df[col].isnull()).sum()
        sh= df.shape[0]
        print("\n")
        print(col)
        print(nul)
        print("Soit: {}% de valeurs None".format((nul/sh)*100))
    pass

In [ ]:
verification(df)

In [ ]:
df.shape

In [ ]:
#on enregistre le dataset contenant les url
export_df= df.to_csv("data/dataset_url_lyrics.csv", index= False, header= True)

## Transformation de nos URL en lyrics

In [ ]:
df= pd.read_csv('data/dataset_url_lyrics.csv')
print(df.shape)
df.head()

In [ ]:
def pretransformation_lyric(url_son):
    """
    
    """
    software_names = [SoftwareName.CHROME.value]
    operating_systems = [OperatingSystem.WINDOWS.value, OperatingSystem.LINUX.value]
    user_agent_rotator = UserAgent(software_names=software_names,
                               operating_systems=operating_systems, limit=100)
    
    random_proxy= sorted(proxies_list, key=lambda x: random())
    for prox in random_proxy:
        try:
            proxies= {"http":transform_proxy_http(prox),
                    "https":transform_proxy_http(prox)}

            user_agent = user_agent_rotator.get_random_user_agent()
            headers= {"User-Agent":user_agent}

            r = requests.Session()
            r.headers.update(headers)
            r.proxies.update(proxies)

            page= r.get(url_son, proxies= proxies, headers= headers)

#             page= requests.get(url_son)
            soup= BeautifulSoup(page.text, 'html.parser')

            lyric= str(page.content)
            pattern= re.compile('(?:Sorry about that. -->)([\s\S]*)(?:-- MxM)')
            res= pattern.search(lyric).group(1)

            #On va transformer notre texte de telle sorte qu'il soit exploitable par nos Lemmnizer / Stemmenizer
            #J'encode en #!utf8 pour accepter les \n etc
            res= res.encode('utf8').decode("unicode_escape")
            #J'y ai mis une balise que j'ai du mal à supprimer
            banword= ['br', 'div', 'brbr']
            #On tokenize notre text
            tokenizer= TreebankWordTokenizer()
            #Je sépare les élément tokenizer
            tokenz= [','.join(tokenizer.tokenize(mot)) for mot in res.split()]

            tokenz= [mot.replace(",", "").replace("<br>", "") for mot in tokenz]
            #J'enlève ce qui n'est pas du texte ou en espace
            tokenz= [re.sub('[^\w\s]', ' ', mot) for mot in tokenz]
            #Je supprime les espaces inutiles
            tokenz= [mot.replace(' ','') for mot in tokenz]
            #Et enfin j'applique ma liste de banword
        #     text_clean= [mot for mot in tokenz if mot not in banword]
            text_clean= ''
            for mot in tokenz:
                if mot not in banword:
                    text_clean += mot + ' '


            return text_clean #sortie de cette facon pour utiliser lemma de spacy

        except:
            print("Mauvais proxy")

In [ ]:
# liste_lyrics= [pretransformation_lyric(url) \
#               for url in df["Url"].values]
# liste_lyrics

In [ ]:
print(df.shape)
df = df.sample(frac=1).reset_index(drop=True)
export_df= df.to_csv("random_dataset_url_lyrics.csv", index= False, header= True)
df.head()

In [ ]:
df["Artiste"].unique()

In [ ]:
#on va scinder notre dataset pour faire plusieurs set de web scraping
df_1= df.iloc[:200,:]
df_2= df.iloc[200:400,:]
df_3= df.iloc[400:600,:]
df_4= df.iloc[600:800,:]
df_5= df.iloc[800:1000,:]
df_6= df.iloc[1000:df.shape[0],:]

In [ ]:
df_1["test"]= df_1.apply(lambda row: pretransformation_lyric(row[3])
                     ,axis= 1)
#pour gérer les erreurs, tester si la page nous a repéré, si oui ==> capcha ou acces denied ?

In [ ]:
print(df_1.shape)
verification(df_1)

In [ ]:
export_df= df_1.to_csv("data/01_dataset_lyrics.csv", index= False, header= True)

In [ ]:
df_2["test"]= df_2.apply(lambda row: pretransformation_lyric(row[3])
                     ,axis= 1)
#pour gérer les erreurs, tester si la page nous a repéré, si oui ==> capcha ou acces denied ?
print(df_2.shape)
verification(df_2)

In [ ]:
export_df= df_2.to_csv("data/02_dataset_lyrics.csv", index= False, header= True)

In [ ]:
df_3["test"]= df_3.apply(lambda row: pretransformation_lyric(row[3])
                     ,axis= 1)
#pour gérer les erreurs, tester si la page nous a repéré, si oui ==> capcha ou acces denied ?
print(df_3.shape)
verification(df_3)

In [ ]:
export_df= df_3.to_csv("data/03_dataset_lyrics.csv", index= False, header= True)

In [ ]:
df= pd.read_csv("data/random_dataset_url_lyrics.csv")

In [ ]:
df_4= df.iloc[600:800,:]
df_5= df.iloc[800:1000,:]
df_6= df.iloc[1000:df.shape[0],:]

In [ ]:
df_4["test"]= df_4.apply(lambda row: pretransformation_lyric(row[3])
                     ,axis= 1)
#pour gérer les erreurs, tester si la page nous a repéré, si oui ==> capcha ou acces denied ?
print(df_4.shape)
verification(df_4)

In [ ]:
export_df= df_4.to_csv("data/04_dataset_lyrics.csv", index= False, header= True)

In [ ]:
df_5["test"]= df_5.apply(lambda row: pretransformation_lyric(row[3])
                     ,axis= 1)
#pour gérer les erreurs, tester si la page nous a repéré, si oui ==> capcha ou acces denied ?
print(df_5.shape)
verification(df_5)

In [ ]:
export_df= df_5.to_csv("data/05_dataset_lyrics.csv", index= False, header= True)

In [ ]:
df_6["test"]= df_6.apply(lambda row: pretransformation_lyric(row[3])
                     ,axis= 1)
#pour gérer les erreurs, tester si la page nous a repéré, si oui ==> capcha ou acces denied ?
print(df_6.shape)
verification(df_6)

In [ ]:
export_df= df_6.to_csv("data/06_dataset_lyrics.csv", index= False, header= True)